In [ ]:
# This notebook produces analysis for outputs from Ocean Plastic Assimilator v0.1
# To run, you will need to install the outputs available at http://doi.org/10.5281/zenodo.4426130
# and install them in an output/ folder at the roor of this repository

In [ ]:
# Retrieve forecast output folders
import os

def is_output_folder(folder_name: str) -> bool:
    return folder_name.find("output_double_gyre_") != -1

folders = []
for f in os.listdir("../outputs/"):
    if is_output_folder(f):
        folders.append("../outputs/" + f)

In [ ]:
import numpy as np
import pandas as pd
from typing import List, Tuple

MAX_ITER_COUNT = 4500

def parseIntTupleList(l: str) -> List[Tuple[float]]:
    content = l.split('[')[1].split(']')[0]
    elements = content.split(', ')
    result = []
    for i in range(0, len(elements), 2):
        x = int(elements[i][1:])
        y = int(elements[i+1][:-1])
        result.append((x,y))

    return result

def retrieveNumValueFromFolderName(f: str, field: str):
    try:
        pair = f.split(field)
        val = pair[1].split('_')[0 if field == 'v' else 1]
        return float(val)
    except ValueError:
        return parseIntTupleList(val)
    except IndexError:
        # Default values for fields that were not defined earlier
        if field == 'eps':
            return 0.25
        if field == 'A':
            return 0.1
        if field == 'nbparts':
            return 25000.0
        return np.NaN

folderNameFields = ['v','as','nbparts','res', 'ens', 'eps', 'rsd', 'A', 'rad', 'devinit', 'meaninit', 'obscov', 'obspoints']
out = {k: [] for k in folderNameFields}
out['lastWMean'] = []
out['lastDErrDev'] = []
out['CFRMS'] = []
out['nbobspoints'] = []

out['iterCount'] = []

for f in folders:
    for k in folderNameFields:
        out[k].append(retrieveNumValueFromFolderName(f, k))
    
    df_sim = pd.read_csv(f + '//log.csv')
    n = df_sim.shape[0] if MAX_ITER_COUNT == None else min(MAX_ITER_COUNT, df_sim.shape[0])
    out['iterCount'].append(n)
    out['lastWMean'].append(df_sim['weights_means'][n-1] if df_sim['weights_means'][n-1] != "--" else np.nan)
    out['lastDErrDev'].append(df_sim['densities_err_stddev'][n-1])
    out['nbobspoints'].append(len(retrieveNumValueFromFolderName(f, 'obspoints')))
    try:
        out['CFRMS'].append(df_sim['CFRMS'][n-1])
    except:
        out['CFRMS'].append(np.nan)

df_lastStats = pd.DataFrame(data=out)

In [ ]:
from sim_vars import LONGITUDES, LATITUDES

df_lastStats = df_lastStats[~np.isnan(df_lastStats['meaninit'])]
df_lastStats['CFRMS'] = df_lastStats['CFRMS'] / np.math.sqrt(LONGITUDES * LATITUDES)
#df_lastStats = df_lastStats[df_lastStats['lastDErrMean'] < 100]
#df_lastStats = df_lastStats[df_lastStats['rad'] == np.inf]

In [ ]:
df_lastStats

# Generate RMSE without correction

In [ ]:
from sim_vars import main_dir_path, LONGITUDES, LATITUDES, MIN_LONGITUDE, MIN_LATITUDE, RESOLUTION
import netCDF4 as nc
import numpy as np

DS_USEFUL_PATH = f"{main_dir_path}/parts_double_gyre_forecast_25000_3.nc"

def computeDensitiesLazy(partsLons, partsLats, partsWeights):
    d = np.zeros((LONGITUDES, LATITUDES))
    for p_id in range(len(partsLons)):
        lonId = int(np.floor((partsLons[p_id] - MIN_LONGITUDE) / RESOLUTION))
        latId = int(np.floor((partsLats[p_id] - MIN_LATITUDE) / RESOLUTION))
        d[min(lonId, LONGITUDES - 1), min(latId, LATITUDES - 1)] += partsWeights[p_id]
    return d

def generateFileName(epsilon, A):
    epsilonStr = str(epsilon).split('.')[0] if str(epsilon)[-1] == '0' else str(epsilon)
    return f"{main_dir_path}parts_double_gyre_ref_eps_{epsilonStr}_A_{A}.nc"

def computeRMSEWithoutCorrection(epsilon, A, muinit):
    ds_uncorrected = nc.Dataset(DS_USEFUL_PATH)
    ds_ref = nc.Dataset(generateFileName(epsilon, A))
    densities_uncorrected = computeDensitiesLazy(ds_uncorrected['lon'][:,-1], ds_uncorrected['lat'][:,-1], ds_uncorrected['weight'][:] * muinit)
    densities_ref = computeDensitiesLazy(ds_ref['lon'][:,-1], ds_ref['lat'][:,-1], ds_ref['weight'][:])
    diff = densities_uncorrected - densities_ref
    RMSE = np.math.sqrt(np.mean(diff ** 2))
    print(RMSE)
    return RMSE

df_lastStats[df_lastStats['v'] == 7.3][df_lastStats['rsd'] == 0][df_lastStats['nbparts'] == 25000].apply(lambda row: computeRMSEWithoutCorrection(row['eps'], row['A'], row['meaninit']), axis=1)

# Generate figures

In [ ]:
out = {}

for f in folders:
    name = ""
    if retrieveNumValueFromFolderName(f, 'rad') != np.inf:
        continue
    if np.isnan(retrieveNumValueFromFolderName(f, 'meaninit')):
        continue
    if retrieveNumValueFromFolderName(f, 'obscov') != 0.01:
        continue
    if len(retrieveNumValueFromFolderName(f, 'obspoints')) != 2:
        continue
    if retrieveNumValueFromFolderName(f, 'ens') != 10:
        continue
    if retrieveNumValueFromFolderName(f, 'nbparts') != 25000:
        continue
    if retrieveNumValueFromFolderName(f, 'v') != 7.3:
        continue
    if retrieveNumValueFromFolderName(f, 'A') != 0.1:
        continue
    if retrieveNumValueFromFolderName(f, 'devinit') != 0.05:
        continue
    for k in ['eps']:
        name += "$\epsilon$ = " + str(retrieveNumValueFromFolderName(f, k))
    df_sim = pd.read_csv(f + '//log.csv')
    n = df_sim.shape[0] if MAX_ITER_COUNT == None else min(MAX_ITER_COUNT, df_sim.shape[0])
    if (df_sim["weights_means"][:] < 5).all():
        out[name] = df_sim["weights_means"]

df_convergence_weight_epsilons_means = pd.DataFrame(out)
df_convergence_weight_epsilons_means = df_convergence_weight_epsilons_means.reindex(sorted(df_convergence_weight_epsilons_means.columns, reverse=True), axis=1)
df_convergence_weight_epsilons_means.insert(0, 'Reference', 1)

In [ ]:
## Figure 6

import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 8))
sns.set_context("poster")
ax = sns.lineplot(data=df_convergence_weight_epsilons_means, dashes=True, color="0.75")
ax.set(xlabel="Iteration count", ylabel="Total mass / $M_{ref}$", ylim=(0,1.5))
#ax.set(title="Evolution of forecast mass over time for different epsilons \nLegend: Double gyre epsilon")

In [ ]:
out = {}

for f in folders:
    name = ""
    if retrieveNumValueFromFolderName(f, 'rad') != np.inf:
        continue
    if np.isnan(retrieveNumValueFromFolderName(f, 'meaninit')):
        continue
    if retrieveNumValueFromFolderName(f, 'obscov') != 0.01:
        continue
    if len(retrieveNumValueFromFolderName(f, 'obspoints')) != 2:
        continue
    if retrieveNumValueFromFolderName(f, 'ens') != 10:
        continue
    if retrieveNumValueFromFolderName(f, 'eps') != 0.25:
        continue
    if retrieveNumValueFromFolderName(f, 'meaninit') != 2:
        continue
    if retrieveNumValueFromFolderName(f, 'nbparts') != 25000:
        continue
    if retrieveNumValueFromFolderName(f, 'v') != 7.3:
        continue
    if retrieveNumValueFromFolderName(f, 'devinit') != 0.05:
        continue
    if retrieveNumValueFromFolderName(f, 'A') not in [0.1, 0.105, 0.11, 0.1175, 0.125]:
        continue
    for k in ['A']:
        name += "$A$ = " + str(retrieveNumValueFromFolderName(f, k))
    df_sim = pd.read_csv(f + '//log.csv')
    n = df_sim.shape[0] if MAX_ITER_COUNT == None else min(MAX_ITER_COUNT, df_sim.shape[0])
    if (df_sim["weights_means"][:] < 5).all():
        out[name] = df_sim["weights_means"]

df_convergence_weight_As_means = pd.DataFrame(out)
df_convergence_weight_As_means = df_convergence_weight_As_means.reindex(sorted(df_convergence_weight_As_means.columns), axis=1)
df_convergence_weight_As_means.insert(0, 'Reference', 1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 8))
sns.set_context("talk")
ax = sns.lineplot(data=df_convergence_weight_As_means, dashes=True, color="0.75")
ax.set(xlabel="Iteration count", ylabel="Total mass relative to $M_{ref}$", ylim=(0,1.5))
# ax.set(title="Evolution of forecast mass over time for different As \nLegend: Double gyre A")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1,figsize=(20, 14))
sns.set_context("talk")
sns.lineplot(data=df_convergence_weight_As_means, dashes=True, color="0.75", ax=axes[1], palette=sns.color_palette(n_colors=6, desat=False))
axes[1].set(xlabel="Iteration count", ylabel="Total mass / $M_{ref}$")
axes[1].set(xscale="linear", ylim=(0,1.5))
sns.lineplot(data=df_convergence_weight_epsilons_means, dashes=True, color="0.75", ax=axes[0], palette=sns.color_palette(n_colors=6, desat=False))
axes[0].set(xlabel="Iteration count", ylabel="Total mass / $M_{ref}$")
axes[0].set(xscale="linear", ylim=(0,1.5))

fig.savefig("figure6.png")

# ax.set(title="Evolution of forecast mass over time for different As \nLegend: Double gyre A")

In [ ]:
out = {}

for f in folders:
    name = ""
    if retrieveNumValueFromFolderName(f, 'rad') != np.inf:
        continue
    if np.isnan(retrieveNumValueFromFolderName(f, 'meaninit')):
        continue
    if retrieveNumValueFromFolderName(f, 'obscov') != 0.01:
        continue
    if retrieveNumValueFromFolderName(f, 'ens') != 10:
        continue
    if retrieveNumValueFromFolderName(f, 'eps') != 0.25:
        continue
    if retrieveNumValueFromFolderName(f, 'A') != 0.1:
        continue
    if retrieveNumValueFromFolderName(f, 'devinit') != 0.05:
        continue
    if retrieveNumValueFromFolderName(f, 'v') != 7.3:
        continue
    if retrieveNumValueFromFolderName(f, 'as') != 3:
        continue
    if retrieveNumValueFromFolderName(f, 'meaninit') not in [0.25,0.5,1,2,5]:
        continue
    for k in ['meaninit']:
        name += "$\mu$ = " + str(retrieveNumValueFromFolderName(f, k))
    df_sim = pd.read_csv(f + '//log.csv')
    n = df_sim.shape[0] if MAX_ITER_COUNT == None else min(MAX_ITER_COUNT, df_sim.shape[0])
    if (df_sim["weights_means"][:] < 10).all():
        out[name] = df_sim["weights_means"]

df_convergence_weight_meaninits_means = pd.DataFrame(out)
df_convergence_weight_meaninits_means['Reference'] = 1
df_convergence_weight_meaninits_means = df_convergence_weight_meaninits_means.reindex(sorted(df_convergence_weight_meaninits_means.columns, reverse=True), axis=1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 8))
sns.set_context("talk")
ax.set(xscale="linear", ylim=(0,1.5))
ax = sns.lineplot(data=df_convergence_weight_meaninits_means, dashes=True, color="0.75", palette=sns.color_palette(n_colors=6, desat=False))
ax.set(xlabel="Iteration count", ylabel="Total mass / $M_{ref}$")
# ax.set(title="Evolution of forecast mass over time for different inital masses \nLegend: Initial total mass relative to reference")
fig.savefig("figure3.png")

In [ ]:
out = {}

for f in folders:
    name = ""
    if retrieveNumValueFromFolderName(f, 'rad') != np.inf:
        continue
    if np.isnan(retrieveNumValueFromFolderName(f, 'meaninit')):
        continue
    if retrieveNumValueFromFolderName(f, 'obscov') not in [0.01]:
        continue
    if retrieveNumValueFromFolderName(f, 'nbparts') not in [25000]:
        continue
    if retrieveNumValueFromFolderName(f, 'v') != 7.3:
        continue
    if retrieveNumValueFromFolderName(f, 'ens') not in [10]:
        continue
    if retrieveNumValueFromFolderName(f, 'meaninit') != 2:
        continue
    if retrieveNumValueFromFolderName(f, 'A') != 0.1:
        continue
    if retrieveNumValueFromFolderName(f, 'eps') != 0.25:
        continue
    if retrieveNumValueFromFolderName(f, 'devinit') != 0.05:
        continue
    if retrieveNumValueFromFolderName(f, 'obspoints')[0] != (12,4):
        continue
    for k in ['obspoints']:
        name += '$N_{o}' + " = " + str(len(retrieveNumValueFromFolderName(f, k))) + "$ "
    df_sim = pd.read_csv(f + '//log.csv')
    n = df_sim.shape[0] if MAX_ITER_COUNT == None else min(MAX_ITER_COUNT, df_sim.shape[0])
    if (df_sim["weights_means"][:] < 5).all():
        nbparts = retrieveNumValueFromFolderName(f, 'nbparts')
        out[name] = df_sim["weights_means"] * nbparts / 25000

df_convergence_weight_No_means = pd.DataFrame(out)
df_convergence_weight_No_means['Reference'] = 1
df_convergence_weight_No_means = df_convergence_weight_No_means.reindex(sorted(df_convergence_weight_No_means.columns, reverse=True), axis=1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 8))
sns.set_context("talk")
ax.set(xscale="linear", ylim=(0.5,1.5))
ax = sns.lineplot(data=df_convergence_weight_No_means, dashes=True, color="0.75", palette=sns.color_palette(n_colors=6, desat=False))
ax.set(xlabel="Iteration count", ylabel="Total mass / $M_{ref}$")
# ax.set(title="Evolution of forecast mass over time for different inital masses \nLegend: Initial total mass relative to reference")
# fig.savefig("figure7.png")

In [ ]:
out = {}

for f in folders:
    name = ""
    if retrieveNumValueFromFolderName(f, 'rad') != np.inf:
        continue
    if np.isnan(retrieveNumValueFromFolderName(f, 'meaninit')):
        continue
    if retrieveNumValueFromFolderName(f, 'ens') not in [10]:
        continue
    if retrieveNumValueFromFolderName(f, 'eps') != 0.25:
        continue
    if retrieveNumValueFromFolderName(f, 'A') != 0.1:
        continue
    if retrieveNumValueFromFolderName(f, 'v') not in [6, 6.1]:
        continue
    for k in ['meaninit', 'obscov', 'obspoints', 'v']:
        name += k + " = " + str(retrieveNumValueFromFolderName(f, k)) + " "
    df_sim = pd.read_csv(f + '//log.csv')
    n = df_sim.shape[0] if MAX_ITER_COUNT == None else min(MAX_ITER_COUNT, df_sim.shape[0])
    if (df_sim["weights_means"][:] < 10).all():
        out[name] = df_sim["weights_means"]

df_convergence_weight_version_means = pd.DataFrame(out)
df_convergence_weight_version_means['Reference'] = 1
df_convergence_weight_version_means = df_convergence_weight_version_means.reindex(sorted(df_convergence_weight_version_means.columns, reverse=True), axis=1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 8))
sns.set_context("talk")
ax.set(xscale="linear", ylim=(0.5,1.2))
ax = sns.lineplot(data=df_convergence_weight_version_means, dashes=False, color="0.75")
ax.set(xlabel="Iteration count", ylabel="Total mass / $M_{ref}$")
# ax.set(title="Evolution of forecast mass over time for different inital masses \nLegend: Initial total mass relative to reference"
# fig.savefig("figurex.png")

# Generate LateX table

In [ ]:
df_paper = df_lastStats[df_lastStats['v'] == 7.3][df_lastStats['rsd'] == 0]
df_paper = df_paper[df_paper['nbparts'] == 25000]
df_paper = df_paper[df_paper['A'] ]
df_paper['RMSE'] = df_paper['CFRMS'].round(decimals=3)
df_paper['FTM'] = df_paper['lastWMean'].round(decimals=3)
df_paper['A'] = df_paper['A'].astype(str)
df_paper['eps'] = df_paper['eps'].astype(str)
df_paper = df_paper[['A', 'eps', 'meaninit', 'devinit', 'nbobspoints', 'obscov', 'FTM', 'RMSE']]

In [ ]:
df_paper['RMSE_NO_DA'] = df_paper.apply(lambda row: computeRMSEWithoutCorrection(row['eps'], row['A'], row['meaninit']), axis=1)

In [ ]:
df_paper['RMSE_NO_DA'] = df_paper['RMSE_NO_DA'].round(decimals=3)

In [ ]:
print(df_paper.sort_values('FTM', ascending=False).to_latex(index=False))